In [1]:
import numpy as np
from copy import copy
import matplotlib.pyplot as plt
from hmmestimate import HMM

/Users/marvinbertin/anaconda/envs/TensorFlow-env/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def generate_HMM_observation(nSteps, pi, T, E):

    def drawFrom(probs):
        return np.where(np.random.multinomial(1,probs) == 1)[0][0]

    observations = np.zeros(nSteps)
    states = np.zeros(nSteps)
    states[0] = drawFrom(pi)
    observations[0] = drawFrom(B[states[0],:])
    for t in range(1,nSteps):
        states[t] = drawFrom(A[states[t-1],:])
        observations[t] = drawFrom(B[states[t],:])
    return observations,states

In [5]:
True_pi = np.array([0.3, 0.7])
True_T = np.array([[0.15, 0.85],
                  [0.88, 0.12]])

In [87]:
# True_E = np.array([[0.6, 0.4],
#                    [0.2, 0.3],
#                    [0.2, 0.3]])
True_E = np.array([[ 0.6,  0.2,  0.2],
                   [ 0.4,  0.3,  0.3]])

In [88]:
True_E

array([[ 0.6,  0.2,  0.2],
       [ 0.4,  0.3,  0.3]])

In [89]:
num_obs = 4
obs, states = generate_HMM_observation(num_obs, True_pi, True_T, True_E)
print obs, states

[ 0.  2.  0.  2.] [ 0.  1.  0.  1.]


/Users/marvinbertin/anaconda/envs/TensorFlow-env/lib/python2.7/site-packages/ipykernel/__main__.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/marvinbertin/anaconda/envs/TensorFlow-env/lib/python2.7/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/marvinbertin/anaconda/envs/TensorFlow-env/lib/python2.7/site-packages/ipykernel/__main__.py:12: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [68]:
# obs = np.array([ 0.,  2.,  2.,  1.])
# states = np.array([ 0.,  1.,  0.,  1.])

In [14]:
hmm = HMM()
hmm.pi = np.array([0.5, 0.5])
hmm.A = np.array([[0.85, 0.15],
                  [0.12, 0.88]])
hmm.B = np.array([[0.8, 0.1, 0.1],
                  [0.0, 0.0, 1]])

hmmguess = HMM()
hmmguess.pi = np.array([0.5, 0.5])
hmmguess.A = np.array([[0.5, 0.5],
                       [0.5, 0.5]])
hmmguess.B = np.array([[0.3, 0.3, 0.4],
                       [0.2, 0.5, 0.3]])

In [15]:
o,s = hmm.simulate(1000)
hmmguess.train(o,0.0001,graphics=False)

In [16]:
print("Actual probabilities \n",hmm.pi)
print ('Estimated initial probabilities\n',hmmguess.pi)

print ('Actual state transition probabililities\n',hmm.A)
print ('Estimated state transition probabililities\n',hmmguess.A)

print ('Actual observation probabililities\n',hmm.B)
print ('Estimated observation probabililities\n',hmmguess.B)

('Actual probabilities \n', array([ 0.5,  0.5]))
('Estimated initial probabilities\n', array([  1.00000000e+00,   1.67160414e-83]))
('Actual state transition probabililities\n', array([[ 0.85,  0.15],
       [ 0.12,  0.88]]))
('Estimated state transition probabililities\n', array([[ 0.86644314,  0.13355686],
       [ 0.10979206,  0.89020794]]))
('Actual observation probabililities\n', array([[ 0.8,  0.1,  0.1],
       [ 0. ,  0. ,  1. ]]))
('Estimated observation probabililities\n', array([[  7.69215318e-01,   1.01493088e-01,   1.29291594e-01],
       [  9.65596634e-04,   2.79889770e-04,   9.98754514e-01]]))


In [23]:
nStates = True_T.shape[0]
nSamples = len(obs)

A = True_T
B = True_E
pi = copy(True_pi)

In [72]:
# alpha_t(i) = P(O_1 O_2 ... O_t, q_t = S_i | hmm)
# Initialize alpha
alpha = np.zeros((nStates,nSamples))
c = np.zeros(nSamples) #scale factors
alpha[:,0] = pi.T * B[:,int(obs[0])]
c[0] = 1.0/np.sum(alpha[:,0])
alpha[:,0] = c[0] * alpha[:,0]
# Update alpha for each observation step
for t in range(1,nSamples):
    alpha[:,t] = np.dot(alpha[:,t-1].T, A).T * B[:,int(obs[t])]
    c[t] = 1.0/np.sum(alpha[:,t])
    alpha[:,t] = c[t] * alpha[:,t]

# Initialize beta
beta = np.zeros((nStates,nSamples))
beta[:,nSamples-1] = 1
beta[:,nSamples-1] = c[nSamples-1] * beta[:,nSamples-1]
# Update beta backwards from end of sequence
for t in range(len(obs)-1,0,-1):
    beta[:,t-1] = np.dot(A, (B[:,int(obs[t])] * beta[:,t]))
    beta[:,t-1] = c[t-1] * beta[:,t-1]
    
xi = np.zeros((nStates,nStates,nSamples-1));
for t in range(nSamples-1):
    denom = np.dot(np.dot(alpha[:,t].T, A) * B[:,int(obs[t+1])].T,
                   beta[:,t+1])
    
    for i in range(nStates):
        numer = alpha[i,t] * A[i,:] * B[:,int(obs[t+1])].T * beta[:,t+1].T
        xi[i,:,t] = numer / denom

# gamma_t(i) = P(q_t = S_i | O, hmm)
gamma = np.squeeze(np.sum(xi,axis=1))
# Need final gamma element for new B
prod =  (alpha[:,nSamples-1] * beta[:,nSamples-1]).reshape((-1,1))
gamma = np.hstack((gamma,  prod / np.sum(prod))) #append one more to gamma!!!

newpi = gamma[:,0]
newA = np.sum(xi,2) / np.sum(gamma[:,:-1],axis=1).reshape((-1,1))
newB = copy(B)

numLevels = B.shape[1]
sumgamma = np.sum(gamma,axis=1)
for lev in range(numLevels):
    mask = obs == lev
    newB[:,lev] = np.sum(gamma[:,mask],axis=1) / sumgamma